In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
df = pd.read_csv('estat_prc_hicp_manr.csv')
df.columns = df.columns.str.strip()
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df.head()

,freq,unit,coicop,geo,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,M,RCH_A,AP,AT,:,:,:,:,:,:,...,5.4,5.4,5.4,5.7,5.8,5.6,4.9,4.8,4.5,:
1,M,RCH_A,AP,BE,:,:,:,:,:,:,...,3.7,3.6,3.7,3.6,3.5,3.4,3.3,3.0,2.9,:
2,M,RCH_A,AP,BG,:,:,:,:,:,:,...,5.4,4.6,5.2,5.1,4.0,3.7,3.2,3.8,3.2,:
3,M,RCH_A,AP,CH,:,:,:,:,:,:,...,1.2,1.1,1.2,1.2,1.2,1.2,1.1,1.1,1.1,:
4,M,RCH_A,AP,CY,:,:,:,:,:,:,...,-0.9,2.8,2.1,4.5,0.2,0.6,0.4,-1.3,0.1,:


In [3]:
path = '../ESTAT_CODELISTS/*.tsv'
files = glob.glob(path)
dataframes = {file.split('/')[-1].replace('.tsv', ''): pd.read_csv(
    file, sep='\t', on_bad_lines='error') for file in files}

In [4]:
for col in ["freq", "unit", "coicop", "geo"]:
    curr_codelist = dataframes[f"ESTAT_{col.upper()}"]
    df[col] = df[col].map(curr_codelist.set_index('CODE')['Label'])

df.rename(columns={'geo': 'country'}, inplace=True)
df.head()

,freq,unit,coicop,country,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Monthly,Annual rate of change,Administered prices,Austria,:,:,:,:,:,:,...,5.4,5.4,5.4,5.7,5.8,5.6,4.9,4.8,4.5,:
1,Monthly,Annual rate of change,Administered prices,Belgium,:,:,:,:,:,:,...,3.7,3.6,3.7,3.6,3.5,3.4,3.3,3.0,2.9,:
2,Monthly,Annual rate of change,Administered prices,Bulgaria,:,:,:,:,:,:,...,5.4,4.6,5.2,5.1,4.0,3.7,3.2,3.8,3.2,:
3,Monthly,Annual rate of change,Administered prices,Switzerland,:,:,:,:,:,:,...,1.2,1.1,1.2,1.2,1.2,1.2,1.1,1.1,1.1,:
4,Monthly,Annual rate of change,Administered prices,Cyprus,:,:,:,:,:,:,...,-0.9,2.8,2.1,4.5,0.2,0.6,0.4,-1.3,0.1,:


In [5]:
df.replace(": z", np.nan, inplace=True)
df.replace(":", np.nan, inplace=True)

In [6]:
year_cols = [col for col in df.columns if col.isdigit()]

for col in year_cols:
    df[col] = df[col].astype(str).str.extract(r'(\d+)').astype(float)

In [7]:
df.head()

,freq,unit,coicop,country,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Monthly,Annual rate of change,Administered prices,Austria,NaN,NaN,NaN,NaN,NaN,NaN,...,5.4,5.4,5.4,5.7,5.8,5.6,4.9,4.8,4.5,NaN
1,Monthly,Annual rate of change,Administered prices,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,...,3.7,3.6,3.7,3.6,3.5,3.4,3.3,3.0,2.9,NaN
2,Monthly,Annual rate of change,Administered prices,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,...,5.4,4.6,5.2,5.1,4.0,3.7,3.2,3.8,3.2,NaN
3,Monthly,Annual rate of change,Administered prices,Switzerland,NaN,NaN,NaN,NaN,NaN,NaN,...,1.2,1.1,1.2,1.2,1.2,1.2,1.1,1.1,1.1,NaN
4,Monthly,Annual rate of change,Administered prices,Cyprus,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.9,2.8,2.1,4.5,0.2,0.6,0.4,-1.3,0.1,NaN


In [8]:
if df["freq"].nunique() == 1:
    df.drop(columns=["freq"], inplace=True)
if df["unit"].unique() == 1:
    df.drop(columns=["unit"], inplace=True)

In [9]:
df.head()

,unit,coicop,country,1997-01,1997-02,1997-03,1997-04,1997-05,1997-06,1997-07,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Annual rate of change,Administered prices,Austria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.4,5.4,5.4,5.7,5.8,5.6,4.9,4.8,4.5,NaN
1,Annual rate of change,Administered prices,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.7,3.6,3.7,3.6,3.5,3.4,3.3,3.0,2.9,NaN
2,Annual rate of change,Administered prices,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.4,4.6,5.2,5.1,4.0,3.7,3.2,3.8,3.2,NaN
3,Annual rate of change,Administered prices,Switzerland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.2,1.1,1.2,1.2,1.2,1.2,1.1,1.1,1.1,NaN
4,Annual rate of change,Administered prices,Cyprus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.9,2.8,2.1,4.5,0.2,0.6,0.4,-1.3,0.1,NaN


In [10]:
df.to_csv("./clean/estat_prc_hicp_manr.csv", index=False)